In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri Aug 30 15:32:29 2024

@author: Thierry ALLEM
"""
# ************   ECHANGES  PHYSIQUES - MACHINE LEARNING  XGBOOST - VISUALISATIONS DES VALEURS PREDITES ET REELLES 2020-2023 ****************

'\nCreated on Fri Aug 30 15:32:29 2024\n\n@author: Thierry ALLEM\n'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import os
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Importation du fichier contenant les valeurs d'échanges physiques prédites sur les années 2020 à 2023
# Lecture du fichier CSV
df_blackout_xgb_20_23 = pd.read_csv('resultats_prev_xgb_20_23_complet.csv', sep=',',low_memory=False)
# Lecture du fichier CSV
df_blackout_xgb_20_23.head()

,region,date_heure,annee,jour_numero,heure,jour_fractionnel,thermique,nucleaire,eolien,solaire,...,cap_prod_max_bioenergies,bioenergies_p_disp,population,heure_decimale,ech_physiques_pred_global,erreur_absolue_glob,erreur_relative_glob,ech_physiques_pred_seg,erreur_absolue_seg,erreur_relative_seg
0,CENTRE VAL DE LOIRE,2020-01-01 00:00:00,2020,1,00:00:00,1.0,105.0,8013.0,18.0,0.0,...,86.0,86.0,2574863,0.0,-5515.9550,233.955078,4.429290,-5340.7110,58.710938,1.111529
1,HAUTS DE FRANCE,2020-01-01 00:00:00,2020,1,00:00:00,1.0,2030.0,4507.0,1575.0,0.0,...,180.0,180.0,5997734,0.0,-1476.4508,174.549194,-10.572332,-1572.7072,78.292847,-4.742147
2,PAYS DE LA LOIRE,2020-01-01 00:00:00,2020,1,00:00:00,1.0,359.0,0.0,135.0,0.0,...,76.0,76.0,3832120,0.0,3392.1577,133.842285,-3.795867,3441.1926,84.807373,-2.405201
3,OCCITANIE,2020-01-01 00:00:00,2020,1,00:00:00,1.0,94.0,2375.0,217.0,0.0,...,146.0,146.0,5973969,0.0,664.7884,222.211609,-25.052042,693.2010,193.799011,-21.848818
4,BOURGOGNE FRANCHE COMTE,2020-01-01 00:00:00,2020,1,00:00:00,1.0,181.0,0.0,247.0,0.0,...,69.0,69.0,2801695,0.0,1992.9108,144.089233,-6.742594,1969.2260,167.774048,-7.850915


In [4]:
# Suppression des colonnes non utiles aux visualisations
col_to_drop = ['thermique','nucleaire','eolien','solaire','hydraulique','bioenergies','pompage','stockage_batterie','destockage_batterie',
                    'cap_prod_max_thermique','cap_prod_max_nucleaire','eolien_p_disp','solaire_p_disp','cap_prod_max_hydraulique',
                    'cap_prod_max_bioenergies',
                    'thermique_p_disp',
                    'nucleaire_p_disp', 'hydraulique_p_disp','bioenergies_p_disp']
df_blackout_xgb_20_23 = df_blackout_xgb_20_23.drop(col_to_drop, axis=1)

df_blackout_xgb_20_23.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649152 entries, 0 to 649151
Data columns (total 15 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   region                     649152 non-null  object 
 1   date_heure                 649152 non-null  object 
 2   annee                      649152 non-null  int64  
 3   jour_numero                649152 non-null  int64  
 4   heure                      649152 non-null  object 
 5   jour_fractionnel           649152 non-null  float64
 6   ech_physiques              649152 non-null  float64
 7   population                 649152 non-null  int64  
 8   heure_decimale             649152 non-null  float64
 9   ech_physiques_pred_global  649152 non-null  float64
 10  erreur_absolue_glob        649152 non-null  float64
 11  erreur_relative_glob       649111 non-null  float64
 12  ech_physiques_pred_seg     649152 non-null  float64
 13  erreur_absolue_seg         64

In [5]:
# Conversion de la colonne 'date_heure 'en Datetime
df_blackout_xgb_20_23['date_heure'] = pd.to_datetime(df_blackout_xgb_20_23['date_heure'])

# Ajout d'une colonne mois
df_blackout_xgb_20_23.insert(4, 'mois', df_blackout_xgb_20_23['date_heure'].dt.month)

# Création d'une colonne date au format YYYY-MM, type Object

df_blackout_xgb_20_23.insert(6, 'mois_annee', df_blackout_xgb_20_23['date_heure'].dt.strftime('%m-%Y'))

In [6]:
# Ajout de colonnes attribuant un indice de confiance aux prévisions

# On définit une petite valeur epsilon pour éviter la division par zéro
epsilon = 1e-10

# Définition des indices de confiance
df_blackout_xgb_20_23['indice_confiance_glob'] = np.where(
    df_blackout_xgb_20_23['ech_physiques'] == 0,
    0,  # Indice de confiance de 0 si la valeur réelle est nulle
    np.maximum(0, 1 - df_blackout_xgb_20_23['erreur_absolue_glob'] / (np.abs(df_blackout_xgb_20_23['ech_physiques']) + epsilon))*100
)

df_blackout_xgb_20_23['indice_confiance_seg'] = np.where(
    df_blackout_xgb_20_23['ech_physiques'] == 0,
    0,  # Indice de confiance de 0 si la valeur réelle est nulle
    np.maximum(0, 1 - df_blackout_xgb_20_23['erreur_absolue_seg'] / (np.abs(df_blackout_xgb_20_23['ech_physiques']) + epsilon))*100
)

df_blackout_xgb_20_23.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649152 entries, 0 to 649151
Data columns (total 19 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   region                     649152 non-null  object        
 1   date_heure                 649152 non-null  datetime64[ns]
 2   annee                      649152 non-null  int64         
 3   jour_numero                649152 non-null  int64         
 4   mois                       649152 non-null  int32         
 5   heure                      649152 non-null  object        
 6   mois_annee                 649152 non-null  object        
 7   jour_fractionnel           649152 non-null  float64       
 8   ech_physiques              649152 non-null  float64       
 9   population                 649152 non-null  int64         
 10  heure_decimale             649152 non-null  float64       
 11  ech_physiques_pred_global  649152 non-null  float64 

In [7]:
# Réorganisation des colonnes
ordre_desire = ['region','date_heure','annee','mois_annee','mois','jour_numero','jour_fractionnel','ech_physiques','ech_physiques_pred_global',
                'erreur_absolue_glob','erreur_relative_glob','indice_confiance_glob', 'ech_physiques_pred_seg','erreur_absolue_seg',
                'erreur_relative_seg','indice_confiance_seg']
df_blackout_xgb_20_23 = df_blackout_xgb_20_23[ordre_desire]

df_blackout_xgb_20_23.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649152 entries, 0 to 649151
Data columns (total 16 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   region                     649152 non-null  object        
 1   date_heure                 649152 non-null  datetime64[ns]
 2   annee                      649152 non-null  int64         
 3   mois_annee                 649152 non-null  object        
 4   mois                       649152 non-null  int32         
 5   jour_numero                649152 non-null  int64         
 6   jour_fractionnel           649152 non-null  float64       
 7   ech_physiques              649152 non-null  float64       
 8   ech_physiques_pred_global  649152 non-null  float64       
 9   erreur_absolue_glob        649152 non-null  float64       
 10  erreur_relative_glob       649111 non-null  float64       
 11  indice_confiance_glob      649152 non-null  float64 

In [8]:
# Recherche de valeurs manquantes
df_blackout_xgb_20_23.isna().sum()

region                        0
date_heure                    0
annee                         0
mois_annee                    0
mois                          0
jour_numero                   0
jour_fractionnel              0
ech_physiques                 0
ech_physiques_pred_global     0
erreur_absolue_glob           0
erreur_relative_glob         41
indice_confiance_glob         0
ech_physiques_pred_seg        0
erreur_absolue_seg            0
erreur_relative_seg          41
indice_confiance_seg          0
dtype: int64

In [9]:
# Brèves statistiques
describe =  df_blackout_xgb_20_23.describe()

In [10]:
# VISUALISATION GRAPHIQUE DES PREDICTIONS - VALEURS DES ECHANGES REELS ET PREDITES SELON LES 2 METHODES ET ERREURS ABSOLUES

In [11]:
# Import des métriques des modélisations régionales (spar segmentation), par année
df_metric_seg_annee = pd.read_csv('resultats_xgb_seg_metrics_20_23.csv', sep=';', encoding='latin-1')
df_metric_seg_annee.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   region  48 non-null     object 
 1   annee   48 non-null     int64  
 2   MSE     48 non-null     float64
 3   RMSE    48 non-null     float64
 4   MAE     48 non-null     float64
 5   R²      48 non-null     float64
dtypes: float64(4), int64(1), object(1)
memory usage: 2.4+ KB


In [12]:
# Création des colonnes vides pour les nouvelles métriques
df_metric_seg_annee['mediane_ech_phys'] = np.nan
df_metric_seg_annee['irq_ech_phys'] = np.nan


df_metric_seg_annee['mediane_ech_phys_glob'] = np.nan
df_metric_seg_annee['irq_ech_phys_glob'] = np.nan

df_metric_seg_annee['mae_err_abs_glob'] = np.nan
df_metric_seg_annee['ecart_type_err_abs_glob'] = np.nan
df_metric_seg_annee['variance_err_abs_glob'] = np.nan

df_metric_seg_annee['mediane_erreur_absolue_glob'] = np.nan
df_metric_seg_annee['irq_err_abs_glob'] = np.nan

df_metric_seg_annee['mae_indice_confiance_global'] = np.nan
df_metric_seg_annee['mediane_indice_confiance_global'] = np.nan

df_metric_seg_annee['mediane_ech_phys_seg'] = np.nan
df_metric_seg_annee['irq_ech_phys_seg'] = np.nan

df_metric_seg_annee['mae_err_abs_seg'] = np.nan
df_metric_seg_annee['ecart_type_err_abs_seg'] = np.nan
df_metric_seg_annee['variance_err_abs_seg'] = np.nan

df_metric_seg_annee['mediane_erreur_absolue_seg'] = np.nan
df_metric_seg_annee['irq_err_abs_seg'] = np.nan

df_metric_seg_annee['mae_indice_confiance_seg'] = np.nan
df_metric_seg_annee['mediane_indice_confiance_seg'] = np.nan

In [13]:
# Fonction pour ajouter des barres de saisons
def barre_saisons(ax):
    saisons = {'Hiver': (1, 79),
               'Printemps': (80, 171),
               'Été': (172, 265),
               'Automne': (266, 354),
               'Hiver (suite)': (355, 365)}
    colors = ['lightblue', 'lightgreen', 'lightyellow', 'orange', 'lightblue']
    for (saison, (start, end)), color in zip(saisons.items(), colors):
        ax.axvspan(start, end, color=color, alpha=0.3)
        ax.text((start + end) / 2, 1.00, saison, ha='center', va='bottom', fontsize=8, color='gray', transform=ax.get_xaxis_transform())

In [14]:
# Création du répertoire de sauvegarde des graphiques
output_dir = "Ech_Phys_XGB_combined_graphs_Ind_Conf"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [15]:
for region in df_blackout_xgb_20_23['region'].unique():
    df_region = df_blackout_xgb_20_23[df_blackout_xgb_20_23['region'] == region]
    
    for annee in df_region['annee'].unique():
        df_annee = df_region[df_region['annee'] == annee]
        
        # Calcul des métriques
        mediane_ech_phys = df_annee['ech_physiques'].median()
        irq_ech_phys = stats.iqr(df_annee['ech_physiques'])
                
        mediane_ech_phys_glob = df_annee['ech_physiques_pred_global'].median()
        irq_ech_phys_glob = stats.iqr(df_annee['ech_physiques_pred_global'])
        
        mediane_ech_phys_seg = df_annee['ech_physiques_pred_seg'].median()
        irq_ech_phys_seg = stats.iqr(df_annee['ech_physiques_pred_seg'])
        
        mae_err_abs_glob = df_annee['erreur_absolue_glob'].mean()
        mediane_erreur_absolue_glob = (np.abs(df_annee['erreur_absolue_glob'])).median()
        irq_err_abs_glob = stats.iqr(df_annee['erreur_absolue_glob']) 
        ecart_type_err_abs_glob = df_annee['erreur_absolue_glob'].std()
        variance_err_abs_glob = df_annee['erreur_absolue_glob'].var()
        
        mae_err_abs_seg = df_annee['erreur_absolue_seg'].mean()
        mediane_erreur_absolue_seg = (np.abs(df_annee['erreur_absolue_seg'])).median()
        irq_err_abs_seg = stats.iqr(df_annee['erreur_absolue_seg']) 
        ecart_type_err_abs_seg = df_annee['erreur_absolue_seg'].std()
        variance_err_abs_seg = df_annee['erreur_absolue_seg'].var()
        
        # Moyennes et médianes des indices de confiance
        mae_indice_confiance_global = df_annee['indice_confiance_glob'].mean()
        mae_indice_confiance_seg = df_annee['indice_confiance_seg'].mean()
        mediane_indice_confiance_global = df_annee['indice_confiance_glob'].median()
        mediane_indice_confiance_seg = df_annee['indice_confiance_seg'].median()
        
        # Mise à jour du DataFrame avec les nouvelles métriques
        df_metric_seg_annee.loc[
            (df_metric_seg_annee['region'] == region) & (df_metric_seg_annee['annee'] == annee),
            ['mediane_ech_phys', 'irq_ech_phys', 'mediane_ech_phys_glob', 'irq_ech_phys_glob','mae_err_abs_glob',
             'ecart_type_err_abs_glob', 'variance_err_abs_glob','mediane_erreur_absolue_glob', 'irq_err_abs_glob',
             'mae_indice_confiance_global', 'mediane_indice_confiance_glob',
             'mediane_ech_phys_seg', 'irq_ech_phys_seg','mae_err_abs_seg', 'ecart_type_err_abs_seg', 'variance_err_abs_seg',
             'mediane_erreur_absolue_seg', 'irq_err_abs_seg', 'mae_indice_confiance_seg', 'mediane_indice_confiance_seg']
            ] = [mediane_ech_phys, irq_ech_phys, mediane_ech_phys_glob, irq_ech_phys_glob,
                 mae_err_abs_glob, ecart_type_err_abs_glob, variance_err_abs_glob,
                 mediane_erreur_absolue_glob, irq_err_abs_glob, mae_indice_confiance_global, 
                 mediane_indice_confiance_global, mediane_ech_phys_seg, irq_ech_phys_seg,
                 mae_err_abs_seg, ecart_type_err_abs_seg, variance_err_abs_seg, mediane_erreur_absolue_seg,
                 irq_err_abs_seg, mae_indice_confiance_seg, mediane_indice_confiance_seg]

        
            # Création du graphique
        fig, axs = plt.subplots(4, 2, figsize=(15, 15))

        # 1ère ligne, 1ère colonne
        axs[0, 0].plot(df_annee['jour_fractionnel'], df_annee['ech_physiques'], color='red', label=f'Ech.phys.-Med= {mediane_ech_phys:.0f} MW')
        axs[0, 0].plot(df_annee['jour_fractionnel'], df_annee['ech_physiques_pred_global'], color='yellow', label=f'Ech.phys.préd.glob.-Med= {mediane_ech_phys_glob:.0f} MW')
        axs[0, 0].set_title("Ech. physiques - prévisions -Modèle global", y = 1.05)
        axs[0, 0].set_xlabel("Jour fractionnel")
        axs[0, 0].set_ylabel("Échanges physiques (MW)")
        axs[0, 0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2)
        barre_saisons(axs[0, 0])  # Ajout des barres de saisons
        
        # 1ère ligne, 2ème colonne
        axs[0, 1].plot(df_annee['jour_fractionnel'], df_annee['ech_physiques'], color='red', label=f'Ech.phys.-Med.= {mediane_ech_phys:.0f} MW')
        axs[0, 1].plot(df_annee['jour_fractionnel'], df_annee['ech_physiques_pred_seg'], color='blue', label=f'Ech.phys.préd.seg.-Med.= {mediane_ech_phys_seg:.0f} MW')
        axs[0, 1].set_title("Ech. physiques - prévision - Modèle régional", y = 1.05)
        axs[0, 1].set_xlabel("Jour fractionnel")
        axs[0, 1].set_ylabel("Échanges physiques (MW)")
        axs[0, 1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2)
        barre_saisons(axs[0, 1])  # Ajout des barres de saisons
        
        # 2ème ligne, 1ère colonne
        window_size = 144
        axs[1, 0].plot(df_annee['jour_numero'], df_annee['ech_physiques'].rolling(window=window_size).mean(), color='red', label='Ech. physiques (moy.mob.)')
        axs[1, 0].plot(df_annee['jour_numero'], df_annee['ech_physiques_pred_global'].rolling(window=window_size).mean(), color='yellow', label='Ech. physiques préd. global (moy.mob)')
        axs[1, 0].set_title("Moyennes mobiles WS144- Modèle global", y = 1.05)
        axs[1, 0].set_xlabel("Jour numero")
        axs[1, 0].set_ylabel("Échanges physiques (MW)")
        axs[1, 0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2)
        barre_saisons(axs[1, 0])  # Ajout des barres de saisons
        
        # 2ème ligne, 2ème colonne
        axs[1, 1].plot(df_annee['jour_numero'], df_annee['ech_physiques'].rolling(window=window_size).mean(), color='red', label='Ech. physiques (moy.mob.)')
        axs[1, 1].plot(df_annee['jour_numero'], df_annee['ech_physiques_pred_seg'].rolling(window=window_size).mean(), color='blue', label='Ech. physiques préd.seg.(moy.mob)')
        axs[1, 1].set_title("Moyennes mobiles WS144 - Modèle régional", y = 1.05)
        axs[1, 1].set_xlabel("Jour numero")
        axs[1, 1].set_ylabel("Échanges physiques (MW)")
        axs[1, 1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2)
        barre_saisons(axs[1, 1])  # Ajout des barres de saisons
                
        # 3ème ligne, 1ère colonne (Erreurs absolues- Préd. global)
        axs[2, 0].plot(df_annee['jour_fractionnel'], df_annee['indice_confiance_glob'], color='yellow', label='indice_confiance_glob.')
        axs[2, 0].axhline(mae_indice_confiance_global, color='red', linestyle='--', label=f'MAE: {mae_indice_confiance_global:.1f} %')
        axs[2, 0].axhline(mediane_indice_confiance_global, color='magenta', linestyle='--', label=f'Med.: {mediane_indice_confiance_global:.1f} %')
        axs[2, 0].set_title("Indices de confiance (100-|erreur relative|) - Modèle global", y = 1.05)
        axs[2, 0].set_xlabel("Jour fractionnel")
        axs[2, 0].set_ylabel("Indice de confiance (%)")
        axs[2, 0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2)
        barre_saisons(axs[2, 0])  # Ajout des barres de saisons
        
        # 3ème ligne, 2ème colonne (Erreurs absolues - Préd. segmenté)
        axs[2, 1].plot(df_annee['jour_fractionnel'], df_annee['indice_confiance_seg'], color='blue', label='indice_confiance_seg')
        axs[2, 1].axhline(mae_indice_confiance_seg, color='red', linestyle='--', label=f'MAE: {mae_indice_confiance_seg:.1f} %')
        axs[2, 1].axhline(mediane_indice_confiance_seg, color='magenta', linestyle='--', label=f'Med.: {mediane_indice_confiance_seg:.1f} %')       
        axs[2, 1].set_title("Indice de confiance (100-|erreur relative|)- Modèle régional", y = 1.05)
        axs[2, 1].set_xlabel("Jour fractionnel")
        axs[2, 1].set_ylabel("Indices de confiance (%)")
        axs[2, 1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2)
        barre_saisons(axs[2, 1])  # Ajout des barres de saisons
 
        # 4ème ligne, 1ère colonne (Distribution des erreurs absolues des 2 méthodes)
        if df_annee['indice_confiance_glob'].var() > 0 and df_annee['indice_confiance_seg'].var() > 0:
            sns.kdeplot(df_annee['indice_confiance_seg'], color='blue', label='indice_confiance_seg', fill=True, ax=axs[3, 0])
            sns.kdeplot(df_annee['indice_confiance_glob'], color='yellow', label='indice_confiance_global', fill=True, alpha = 0.4, ax=axs[3, 0])
   
        else:
            axs[3, 0].text(0.5, 0.5, "Variabilité insuffisante\npour KDE", fontsize=12, ha='center', va='center')

        axs[3, 0].set_title("Distribution des indices de confiance des prévisions")
        axs[3, 0].set_xlabel("Indice de confiance (%)")
        axs[3, 0].set_ylabel("Densité")    
        axs[3, 0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2)
        # 4ème ligne, 2ème colonne (Ajout des métriques de prévisions)
        metrics = df_metric_seg_annee[(df_metric_seg_annee['region'] == region) & (df_metric_seg_annee['annee'] == annee)]
        if not metrics.empty:
            # Texte pour chaque colonne
            col1_text = (
                f"Médiane: {mediane_ech_phys:.0f} MW\n"
                f"IQR: {irq_ech_phys:.0f} MW\n"
                )
            col2_text = (
                f"Médiane: {mediane_ech_phys_glob:.0f} MW\n"
                f"IQR: {irq_ech_phys_glob:.0f} MW\n"
                f"Ind.Conf.Moy.: {mae_indice_confiance_global:.1f} %\n"
                )
            col3_text = (
                f"Médiane: {mediane_ech_phys_seg:.0f} MW\n"
                f"IQR: {irq_ech_phys_seg:.0f} MW\n"
                f"Ind.Conf.Moy.: {mae_indice_confiance_seg:.1f} %\n"
                )

            # Affichage des textes dans les colonnes
            axs[3, 1].text(0.5, 0.93, f"Métriques {annee}", fontsize=14, ha='left', va='bottom', 
                           bbox=dict(facecolor='white', edgecolor='brown', boxstyle='round4,pad=0.3'))
            
            axs[3, 1].text(0.17, 0.7, "Valeurs réelles", fontsize=12, ha='left', va='top')
            axs[3, 1].text(0.5, 0.7, "Modèle global", fontsize=12, ha='left', va='top')
            axs[3, 1].text(0.83, 0.7, "Modèle régional", fontsize=12, ha='left', va='top')
            
            
            # Ajout des résultats
            axs[3, 1].text(0.17, 0.5, col1_text, fontsize=10, ha='left', va='top')
            axs[3, 1].text(0.5, 0.5, col2_text, fontsize=10, ha='left', va='top')
            axs[3, 1].text(0.83, 0.5, col3_text, fontsize=10, ha='left', va='top')
    
        else:
            axs[3, 1].text(0.5, 0.5, "Aucune métrique disponible", fontsize=12, ha='center', va='center') 
        
        # Masquage du sous-graphe vide
        axs[3, 1].axis('off')

        # Ajustement de l'espacement entre les sous-graphes
        plt.subplots_adjust(hspace=0.6, wspace=0.3, top=0.92, bottom=0.08)
        
        # Ajout du titre principal
        fig.suptitle(f"Prévisions des échanges physiques par XGBoost sans transformation cible \n{region}, {annee}", fontsize=16)
        fig.text(0.5,0.01,"XGBoost *subsample: 0.7, n_estimators: 400, max_depth: 6, learning_rate: 0.1, 'gamma': 0.2, 'colsample_bytree': 0.7", ha='center', fontsize = 12)
                
        # Sauvegarde du graphique
        filename = f"{region}_{annee}_combined.png"
        plt.savefig(os.path.join(output_dir, filename), dpi=300)
        plt.close()

In [16]:
df_metric_seg_annee.head()

,region,annee,MSE,RMSE,MAE,R²,mediane_ech_phys,irq_ech_phys,mediane_ech_phys_glob,irq_ech_phys_glob,...,mediane_ech_phys_seg,irq_ech_phys_seg,mae_err_abs_seg,ecart_type_err_abs_seg,variance_err_abs_seg,mediane_erreur_absolue_seg,irq_err_abs_seg,mae_indice_confiance_seg,mediane_indice_confiance_seg,mediane_indice_confiance_glob
0,CENTRE VAL DE LOIRE,2020,235548.1481,485.333028,357.112431,0.921531,-6473.0,2939.25,-6118.0465,2557.337275,...,-6185.82780,2468.490925,360.789356,337.817404,114120.598476,270.296143,383.386963,93.158463,95.345733,95.327329
1,CENTRE VAL DE LOIRE,2021,155054.4326,393.769517,313.251704,0.880647,-6253.0,1622.00,-6079.3962,1655.602800,...,-6038.79935,1595.591800,308.151864,243.413842,59250.298341,261.837158,304.717773,95.141627,95.709284,95.987436
2,CENTRE VAL DE LOIRE,2022,169071.0998,411.182563,309.426938,0.930583,-5357.5,2565.50,-5060.0875,2040.789475,...,-5336.27850,2105.468075,316.143322,285.350774,81425.064398,228.861694,357.137329,94.688753,95.677124,94.333574
3,CENTRE VAL DE LOIRE,2023,538332.2605,733.711292,670.038008,0.559392,-8440.0,1368.75,-7690.5765,939.315250,...,-7809.53385,967.253625,664.424223,302.734865,91648.398680,654.773926,422.380493,92.301746,92.236984,89.942853
4,PAYS DE LA LOIRE,2020,179310.6228,423.450851,341.935977,0.434895,2187.0,787.25,2297.2935,745.911950,...,2397.74500,683.492725,331.889079,248.146711,61576.790223,281.329956,340.399719,83.038711,87.499941,88.023229


In [17]:
df_metric_seg_annee.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 27 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   region                           48 non-null     object 
 1   annee                            48 non-null     int64  
 2   MSE                              48 non-null     float64
 3   RMSE                             48 non-null     float64
 4   MAE                              48 non-null     float64
 5   R²                               48 non-null     float64
 6   mediane_ech_phys                 48 non-null     float64
 7   irq_ech_phys                     48 non-null     float64
 8   mediane_ech_phys_glob            48 non-null     float64
 9   irq_ech_phys_glob                48 non-null     float64
 10  mae_err_abs_glob                 48 non-null     float64
 11  ecart_type_err_abs_glob          48 non-null     float64
 12  variance_err_abs_glob   

In [18]:
# Export du DataFrame avec les métriques
df_metric_seg_annee.to_excel('resultats_xgb_metrics_20_23_glob_seg_IC.xlsx', index=False)
df_metric_seg_annee.to_csv('resultats_xgb_metrics_20_23_glob_seg_IC.csv', index=False)

In [19]:
# ******** Visualisation des médianes annuelles des valeurs réelles des echanges physiques en fonction des médianes des indices de confiance ********

In [20]:
# Filtrage des données par année
annees = df_metric_seg_annee['annee'].unique()

for annee in annees:
    # Création de la figure
    fig, axs = plt.subplots(2, 1, figsize=(15, 24))
     
    df_annee = df_metric_seg_annee[df_metric_seg_annee['annee'] == annee]
    
    # Nuage de points des 'indice_confiance_global'
    scatter1 = sns.scatterplot(ax=axs[0],x='mediane_indice_confiance_global', y='mediane_ech_phys', hue='region', palette='tab10', s=100,
                    data=df_annee, legend = 'brief')
    
    axs[0].set_xlim(0, 110)
    axs[0].set_title("prévisions par  modélisation globale", fontsize=14)
    axs[0].set_xlabel("Médiane Indice Confiance Global (%)")
    axs[0].set_ylabel("Médiane Échanges Physiques (MW)")
    axs[0].grid(True, which='both', linestyle='--', linewidth=0.5, color='grey')
    # Légende
    axs[0].legend(loc='upper center', bbox_to_anchor=(0.5, 1.3), ncol=2, fontsize=12)
    
    # Nuage de points des 'indice_confiance_seg'
    sns.scatterplot(ax=axs[1],x='mediane_indice_confiance_seg', y='mediane_ech_phys',hue='region',palette='tab10',s=100, 
                               data=df_annee, legend = False)
    
    axs[1].set_xlim(0, 110)
    axs[1].set_title("prévisions par modélisations régionales", fontsize=14)
    axs[1].set_xlabel("Médiane Indice Confiance Segmenté (%)")
    axs[1].set_ylabel("Médiane Échanges Physiques (MW)")
    axs[1].grid(True, which='both', linestyle='--', linewidth=0.5, color='grey') 
    
    # Titre général
    fig.suptitle(f"Echanges physiques (Médianes annuelles) & Indices de Confiance de modélisation  (médianes) - {annee}", fontsize=22, weight='bold', y=0.96)

    # Ajustement de la mise en page et sauvegarde de l'image
    plt.tight_layout(rect=[0, 0, 1, 0.93])
    plt.savefig(f"scatters_mediane_ech_phys_IC_{annee}.png", dpi=300, bbox_inches='tight')
    plt.close(fig)

In [21]:
# ************************ Nuages de points de toutes les valeurs 'echanges physiques' réelles en fonction des indices de confiance ***************

In [22]:
# Initialisation d'une palette de couleurs selon la région
palette = sns.color_palette('tab10', n_colors=df_blackout_xgb_20_23['region'].nunique())
annees = df_blackout_xgb_20_23['annee'].unique()
for annee in annees:
    df_annee = df_blackout_xgb_20_23[df_blackout_xgb_20_23['annee'] == annee]
    
    # Création d'une figure avec deux sous-graphiques
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(15, 24))
    
    # Premier sous-graphique : ech_physiques en fonction de 'indice_confiance_glob'
    sns.scatterplot(ax=axes[0], data=df_annee, x='indice_confiance_glob', y='ech_physiques', 
                    hue='region', palette=palette, legend=False, s = 10)
    axes[0].set_title("prévisions par modélisation globale")
    axes[0].set_xlabel('Indice de Confiance glob.(%)')
    axes[0].set_ylabel('Échanges Physiques réels (MW)')

    # Deuxième sous-graphique : ech_physiques en fonction de 'indice_confiance_seg'
    sns.scatterplot(ax=axes[1], data=df_annee, x='indice_confiance_seg', y='ech_physiques', 
                    hue='region', palette=palette, legend=False, s=10)
    axes[1].set_title("prévisions par modélisations régionales")
    axes[1].set_xlabel('Indice de Confiance reg.(%)')
    axes[1].set_ylabel('Échanges Physiques réels (MW)')

    # Légende, sur le 1er sous-graphique
    handles, labels = axes[0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper right', title='Région')
    
    # Titre général
    fig.suptitle(f"Echanges physiques & Indices de Confiance de modélisation- {annee}", fontsize=22, weight='bold', y=0.96)

    # Ajustement de la mise en page pour éviter les chevauchements
    plt.tight_layout()

    # Sauvegarde du graphique
    plt.savefig(f'scatters_ech_physiques_IC{annee}.png', dpi=300)
    
    plt.close()

In [23]:
#  Nuages de points de toutes les valeurs 'echanges physiques' réelles en fonction des indices de confiance, avec affichage des tendances linéaires

In [24]:
# Initialisation d'une palette de couleurs pour chaque région
palette = sns.color_palette('tab10', n_colors=df_blackout_xgb_20_23['region'].nunique())
annees = df_blackout_xgb_20_23['annee'].unique()

for annee in annees:
    df_annee = df_blackout_xgb_20_23[df_blackout_xgb_20_23['annee'] == annee]
    
    # Création d'une figure avec deux sous-graphiques
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(15, 24))
       
    regions = df_annee['region'].unique()

    # Premier sous-graphique (modélisation globale) : ech_physiques en fonction de indice_confiance_glob
    sns.scatterplot(ax=axes[0], data=df_annee, x='indice_confiance_glob', y='ech_physiques', 
                    hue='region', palette=palette, legend=False, s=10, alpha=0.75)  # Pas de légende ici
    axes[0].set_title("prévisions par modélisation globale")
    axes[0].set_xlabel('Indice de Confiance glob.(%)')
    axes[0].set_ylabel('Échanges Physiques réels (MW)')

    # Ajout de courbes de tendance linéaires par région
    for region in regions:
        df_region = df_annee[df_annee['region'] == region]
        sns.regplot(ax=axes[0], data=df_region, x='indice_confiance_glob', y='ech_physiques', 
                    scatter=False, color=palette[regions.tolist().index(region)], 
                    line_kws={'linestyle':'--', 'linewidth':2})

    # Deuxième sous-graphique (modélisation régionale) : ech_physiques en fonction de indice_confiance_seg
    sns.scatterplot(ax=axes[1], data=df_annee, x='indice_confiance_seg', y='ech_physiques', 
                    hue='region', palette=palette, legend=False, s=10, alpha=0.75)  # Pas de légende ici
    axes[1].set_title("prévisions par modélisations régionales")
    axes[1].set_xlabel('Indice de Confiance reg.(%)')
    axes[1].set_ylabel('Échanges Physiques réels (MW)')

    # Ajout de courbes de tendance linéaires par région
    for region in regions:
        df_region = df_annee[df_annee['region'] == region]
        sns.regplot(ax=axes[1], data=df_region, x='indice_confiance_seg', y='ech_physiques', 
                    scatter=False, color=palette[regions.tolist().index(region)], 
                    line_kws={'linestyle':'--', 'linewidth':2})

    # Ajout de la légende globale à la figure entière
    handles, labels = axes[0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=len(regions), title='Région')
    
    # Titre général
    fig.suptitle(f"Echanges physiques & Indices de Confiance de modélisation - {annee}", fontsize=22, weight='bold', y=0.96)

    # Ajustement de la mise en page pour éviter les chevauchements
    plt.tight_layout()

    # Sauvegarde du graphique
    plt.savefig(f'scatters_ech_physiques_IC_TL{annee}.png', dpi=300)
    plt.close()

In [25]:
# Nuages de points de toutes les valeurs 'echanges physiques' réelles en fonction des indices de confiance, avec affichage des tendances -lissage LOWESS

In [26]:
palette = sns.color_palette('tab10', n_colors=df_blackout_xgb_20_23['region'].nunique())
annees = df_blackout_xgb_20_23['annee'].unique()

for annee in annees:
    df_annee = df_blackout_xgb_20_23[df_blackout_xgb_20_23['annee'] == annee]
    
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(15, 24))
    
    regions = df_annee['region'].unique()

    # Premier sous-graphique avec régression polynomiale (ordre 2)
    scatter_0 = sns.scatterplot(ax=axes[0], data=df_annee, x='indice_confiance_glob', y='ech_physiques', 
                    hue='region', palette=palette, legend=False, s=10, alpha=0.6)
    axes[0].set_title("Par modélisation globale")
    axes[0].set_xlabel('Indice de Confiance glob.(%)')
    axes[0].set_ylabel('Échanges Physiques réels (MW)')
    
    # Courbes de tendance polynomiale par région
    for region in regions:
        df_region = df_annee[df_annee['region'] == region]
        sns.regplot(ax=axes[0], data=df_region, x='indice_confiance_glob', y='ech_physiques', 
                    scatter=False, color=palette[regions.tolist().index(region)], 
                    line_kws={'linestyle':'--', 'linewidth':2}, lowess=True)  # Lissage LOWESS

    # Deuxième sous-graphique avec LOWESS
    scatter_1 = sns.scatterplot(ax=axes[1], data=df_annee, x='indice_confiance_seg', y='ech_physiques', 
                    hue='region', palette=palette, legend=False, s=10, alpha=0.6)
    axes[1].set_title("Par modélisations régionales")
    axes[1].set_xlabel('Indice de Confiance reg.(%)')
    axes[1].set_ylabel('Échanges Physiques réels (MW)')
    
    # Courbes de tendance LOWESS par région
    for region in regions:
        df_region = df_annee[df_annee['region'] == region]
        sns.regplot(ax=axes[1], data=df_region, x='indice_confiance_seg', y='ech_physiques', 
                    scatter=False, color=palette[regions.tolist().index(region)], 
                    line_kws={'linestyle':'--', 'linewidth':2}, lowess=True)  # Lissage LOWESS

    # Ajout de la légende commune en utilisant les handles et labels du premier graphique
    handles, labels = scatter_0.get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper center', title='Région', bbox_to_anchor=(0.5, 0.95), ncol=len(regions), fontsize='large')

    # Titre général
    fig.suptitle(f"Echanges physiques & Indices de Confiance de modélisation + LOWESS - {annee}", fontsize=22, weight='bold', y=0.98)

    plt.tight_layout()
    plt.savefig(f'scatters_ech_physiques_IC_Tendances_LLW_{annee}.png', dpi=300)
    plt.close()

In [27]:
# ************************Hstogrammes des bilans annuels des échanges physiques réels et prédits selon les 2 méthodes ****************************

In [28]:
# Calcul des sommes par année et région
somme_values = df_blackout_xgb_20_23.groupby(['region', 'annee'])[['ech_physiques', 'ech_physiques_pred_global', 'ech_physiques_pred_seg']].sum().reset_index()

# Transformation des valeurs de MW en TWh (on divise par 2 car les valeurs sont relevées toutes les 0.5h et on divise 10E6 pour obtenir les TWh)
somme_values[['ech_physiques', 'ech_physiques_pred_global', 'ech_physiques_pred_seg']] /= 1_000_000/2 

# Couleurs des barres
colors = ['red', 'yellow', 'blue']

# Itération sur les années pour créer un graphique par année
for annee in somme_values['annee'].unique():
    # Filtrage des données pour l'année en cours
    df_annee = somme_values[somme_values['annee'] == annee]
    regions = df_annee['region'].unique()
    
    # Création de la figure et des axes
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Paramètres pour les barres
    bar_width = 0.25  # Largeur des barres
    index = np.arange(len(regions))  # Position des groupes sur l'axe y
    
    # Création des barres côte à côte
    ax.barh(index, df_annee['ech_physiques'], bar_width, color=colors[0], label='Ech. physiques')
    ax.barh(index + bar_width, df_annee['ech_physiques_pred_global'], bar_width, color=colors[1], label='Ech. physiques préd. global')
    ax.barh(index + 2 * bar_width, df_annee['ech_physiques_pred_seg'], bar_width, color=colors[2], label='Ech. physiques préd. segmenté')
    
    # Ajout de traits de séparation entre les régions
    for i in range(len(regions) - 1):
        ax.axhline(y=i + 0.75, color='gray', linestyle='--', linewidth=0.5)
    
    # Affichage des valeurs au bout de chaque barre 'ech_physiques' avec ajustement des positions
    for i, v in enumerate(df_annee['ech_physiques']):
        # Décalage horizontal en fonction de la valeur (positif ou négatif)
        horizontal_offset = 0.7 if v > 0 else -30
        ax.text(v + horizontal_offset, i, f'{v:.0f}', color='black', va='center', fontsize=8)
    
    # Affichage du pourcentage de différence pour 'ech_physiques_pred_global' avec décalage
    for i, (v_real, v_pred_global) in enumerate(zip(df_annee['ech_physiques'], df_annee['ech_physiques_pred_global'])):
        percentage_diff_global = (v_pred_global - v_real) / v_real * 100
        horizontal_offset_global = 0.9 if v_pred_global > 0 else -38
        ax.text(v_pred_global + horizontal_offset_global, i + bar_width, f'{percentage_diff_global:.1f}%', color='black', va='center', fontsize=8)
    
    # Affichage du pourcentage de différence pour 'ech_physiques_pred_seg' avec décalage
    for i, (v_real, v_pred_seg) in enumerate(zip(df_annee['ech_physiques'], df_annee['ech_physiques_pred_seg'])):
        percentage_diff_seg = (v_pred_seg - v_real) / v_real * 100
        horizontal_offset_seg = 0.9 if v_pred_seg > 0 else -38
        ax.text(v_pred_seg + horizontal_offset_seg, i + 2 * bar_width, f'{percentage_diff_seg:.1f}%', color='black', va='center', fontsize=8)
    
    # Configuration des légendes, titres et axes
    ax.set_xlabel('Echanges physiques annuels (TWh)')
    ax.set_ylabel('Région')
    ax.set_title(f'Comparaison des échanges physiques annuels réels / prédits (XGBoost) par région, {annee}')
    ax.set_yticks(index + bar_width)
    ax.set_yticklabels(regions)
    
    # Ajustement des limites de l'axe x pour ajouter de l'espace et tenir compte de la plage élargie
    ax.set_xlim(-290, df_annee[['ech_physiques', 'ech_physiques_pred_global', 'ech_physiques_pred_seg']].max().max() * 1.3)
    
    # Ajustement de la légende afin qu'elle ne recouvre pas les graphiques
    ax.legend(loc='upper left', bbox_to_anchor=(1, 1))
    
    plt.tight_layout()
    
    # Sauvegarde du graphique
    plt.savefig(f'comparaison_valeurs_region_{annee}.png', dpi=300)
    plt.close()

In [29]:
print("FIN DU CODE")

FIN DU CODE
